In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

In [33]:
# Make sure it's sorted by city, zipcode, and date
df = df.sort_values(['city', 'zipcode', 'date']).reset_index(drop=True)

# Convert date to datetime if not already
df['date'] = pd.to_datetime(df['date'])


In [34]:
train = df[df.date.dt.year<2023]
#train

In [35]:
test = df[df.date.dt.year>2022]
#test

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 809380 entries, 0 to 884079
Data columns (total 39 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   date                          809380 non-null  datetime64[ns]
 1   median_sale_price             809380 non-null  float64       
 2   median_list_price             809380 non-null  float64       
 3   median_ppsf                   809380 non-null  float64       
 4   median_list_ppsf              809380 non-null  float64       
 5   homes_sold                    809380 non-null  float64       
 6   pending_sales                 809380 non-null  float64       
 7   new_listings                  809380 non-null  float64       
 8   inventory                     809380 non-null  float64       
 9   median_dom                    809380 non-null  float64       
 10  avg_sale_to_list              809380 non-null  float64       
 11  sold_above_list   

In [37]:
# Group by city and zipcode for proper time series handling
grouped = train.groupby(['city', 'zipcode'])

In [51]:
ts_train = pd.DataFrame()

In [52]:
# Create 3-month profitability target
ts_train['target_profitable_3m'] = (grouped['price'].shift(-3) > 1.005 * train['price']).astype(int)

numeric_column_names = train.select_dtypes(include='number').columns

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 6 months
        ts_train[f'{feature}_{lag}m'] = grouped[f'{feature}'].shift(lag)


/tmp/ipykernel_42193/2052503371.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_42193/2052503371.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_42193/2052503371.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

In [53]:
# Remove rows where target cannot be calculated (last 3 months for each location)
ts_train = ts_train.dropna().reset_index(drop=True)


print("Created features:")
print("- target_profitable_3m")
print("- per_capita_income_lag_1m to per_capita_income_lag_6m")
print("- price_lag_1m to price_lag_6m")
print(f"Final dataset shape: {ts_train.shape}")

Created features:
- target_profitable_3m
- per_capita_income_lag_1m to per_capita_income_lag_6m
- price_lag_1m to price_lag_6m
Final dataset shape: (790702, 109)


In [54]:
ts_train['target_profitable_3m'].value_counts()

target_profitable_3m
1    587370
0    203332
Name: count, dtype: int64

In [42]:
ts_train.columns.values

array(['target_profitable_3m', 'median_sale_price_1m',
       'median_sale_price_2m', 'median_sale_price_3m',
       'median_list_price_1m', 'median_list_price_2m',
       'median_list_price_3m', 'median_ppsf_1m', 'median_ppsf_2m',
       'median_ppsf_3m', 'median_list_ppsf_1m', 'median_list_ppsf_2m',
       'median_list_ppsf_3m', 'homes_sold_1m', 'homes_sold_2m',
       'homes_sold_3m', 'pending_sales_1m', 'pending_sales_2m',
       'pending_sales_3m', 'new_listings_1m', 'new_listings_2m',
       'new_listings_3m', 'inventory_1m', 'inventory_2m', 'inventory_3m',
       'median_dom_1m', 'median_dom_2m', 'median_dom_3m',
       'avg_sale_to_list_1m', 'avg_sale_to_list_2m',
       'avg_sale_to_list_3m', 'sold_above_list_1m', 'sold_above_list_2m',
       'sold_above_list_3m', 'off_market_in_two_weeks_1m',
       'off_market_in_two_weeks_2m', 'off_market_in_two_weeks_3m',
       'zipcode_1m', 'zipcode_2m', 'zipcode_3m', 'year_1m', 'year_2m',
       'year_3m', 'bank_1m', 'bank_2m', 'bank_3m

In [43]:
ts_train[ts_train.zipcode_1m==30002]

,target_profitable_3m,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,median_ppsf_3m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,125500.0,61870.0,46550.0,217450.0,245000.0,217450.0,63.913043,40.723982,31.813674,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202681.309539,202421.064584,200773.999557
1,0,153000.0,125500.0,61870.0,189900.0,217450.0,245000.0,81.598080,63.913043,40.723982,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202998.603897,202681.309539,202421.064584
2,0,165500.0,153000.0,125500.0,154000.0,189900.0,217450.0,81.598080,81.598080,63.913043,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,203781.903446,202998.603897,202681.309539
3,0,140250.0,165500.0,153000.0,145000.0,154000.0,189900.0,78.918228,81.598080,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205012.252913,203781.903446,202998.603897
4,0,140000.0,140250.0,165500.0,103950.0,145000.0,154000.0,78.918228,78.918228,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205414.599202,205012.252913,203781.903446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0,602750.0,555000.0,519000.0,542500.0,540000.0,525000.0,234.281690,230.061350,218.750000,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,499412.241077,495606.636012,488436.530983
123,0,568750.0,602750.0,555000.0,455000.0,542500.0,540000.0,232.192450,234.281690,230.061350,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,500013.225078,499412.241077,495606.636012
124,0,570000.0,568750.0,602750.0,472642.5,455000.0,542500.0,225.866425,232.192450,234.281690,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,498003.211865,500013.225078,499412.241077
125,0,572500.0,570000.0,568750.0,460285.0,472642.5,455000.0,231.490967,225.866425,232.192450,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,496887.003144,498003.211865,500013.225078


In [44]:
ts_train.loc[120:135,['price_1m', 'zipcode_1m']]

,price_1m,zipcode_1m
120,488436.530983,30002.0
121,495606.636012,30002.0
122,499412.241077,30002.0
123,500013.225078,30002.0
124,498003.211865,30002.0
125,496887.003144,30002.0
126,495875.243858,30002.0
127,289766.001103,30004.0
128,290093.878929,30004.0
129,290533.730909,30004.0


In [45]:
ts_train

,target_profitable_3m,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,median_ppsf_3m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,125500.0,61870.0,46550.0,217450.0,245000.0,217450.0,63.913043,40.723982,31.813674,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202681.309539,202421.064584,200773.999557
1,0,153000.0,125500.0,61870.0,189900.0,217450.0,245000.0,81.598080,63.913043,40.723982,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202998.603897,202681.309539,202421.064584
2,0,165500.0,153000.0,125500.0,154000.0,189900.0,217450.0,81.598080,81.598080,63.913043,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,203781.903446,202998.603897,202681.309539
3,0,140250.0,165500.0,153000.0,145000.0,154000.0,189900.0,78.918228,81.598080,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205012.252913,203781.903446,202998.603897
4,0,140000.0,140250.0,165500.0,103950.0,145000.0,154000.0,78.918228,78.918228,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205414.599202,205012.252913,203781.903446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790697,0,438000.0,435000.0,435000.0,445250.0,435000.0,450000.0,298.229399,305.238677,299.755501,...,37698.0,37698.0,37698.0,37698.0,15002.0,15002.0,15002.0,430678.116793,425381.408635,416349.153215
790698,0,435500.0,438000.0,435000.0,429350.0,445250.0,435000.0,293.754998,298.229399,305.238677,...,37698.0,37698.0,37698.0,37698.0,15002.0,15002.0,15002.0,432523.250209,430678.116793,425381.408635
790699,0,430000.0,435500.0,438000.0,449999.5,429350.0,445250.0,293.646556,293.754998,298.229399,...,37698.0,37698.0,37698.0,37698.0,15002.0,15002.0,15002.0,431488.991172,432523.250209,430678.116793
790700,0,442000.0,430000.0,435500.0,435000.0,449999.5,429350.0,294.548415,293.646556,293.754998,...,37698.0,37698.0,37698.0,37698.0,15002.0,15002.0,15002.0,429747.893176,431488.991172,432523.250209


SGD Classifier look on the precision on the test set.
Gradient boosting classifier.

XGBoost classifier.

-> Target might be changed to a regression where we predict the price of the zip code.

-> Convert zipcode into longitude and latitude if possioble. -> Advantage: The model can differenciate the distance between the cities.

Scale with min max scaler as always not 